In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = (10, 8)
import collections

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Часть 1. Игрушечный набор данных "девушка в баре"
Цель – "на пальцах", с помощью игрушечной задачи классификации разобраться в том, как работают деревья решений. Само по себе дерево решений – довольно слабый алгоритм, но основанные на нем алгоритмы случайного леса и градиентного бустинга - пожалуй, лучшее, что есть на сегодняшний день (в задачах, где можно обойтись без нейронных сетей). Поэтому разобраться в том, как работает дерево решений, полезно.

Рассмотрим игрушечную задачу бинарной классификации: поедет ли с Вами девушка из бара? Это будет зависеть от Вашей внешности и красноречия, крепости предлагаемых напитков и, как это ни меркантильно, от количества потраченных в баре денег.

In [2]:
# Создание датафрейма с dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
def intersect_features(train, test):
    common_feat = list( set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]

In [3]:
features = ['Внешность', 'Алкоголь_в_напитке',
            'Уровень_красноречия', 'Потраченные_деньги']

#### Обучающая выборка

In [4]:
df_train = {}
df_train['Внешность'] = ['приятная', 'приятная', 'приятная', 'отталкивающая',
                         'отталкивающая', 'отталкивающая', 'приятная'] 
df_train['Алкоголь_в_напитке'] = ['да', 'да', 'нет', 'нет', 'да', 'да', 'да']
df_train['Уровень_красноречия'] = ['высокий', 'низкий', 'средний', 'средний', 'низкий',
                                   'высокий', 'средний']
df_train['Потраченные_деньги'] = ['много', 'мало', 'много', 'мало', 'много',
                                  'много', 'много']
df_train['Поедет'] = LabelEncoder().fit_transform(['+', '-', '+', '-', '-', '+', '+'])

df_train = create_df(df_train, features)
df_train

,Поедет,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_низкий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,0,False,True,True,False,True,False,False,False,True
1,1,False,True,True,False,False,True,False,True,False
2,0,False,True,False,True,False,False,True,False,True
3,1,True,False,False,True,False,False,True,True,False
4,1,True,False,True,False,False,True,False,False,True
5,0,True,False,True,False,True,False,False,False,True
6,0,False,True,True,False,False,False,True,False,True


#### Тестовая выборка

In [5]:
df_test = {}
df_test['Внешность'] = ['приятная', 'приятная', 'отталкивающая'] 
df_test['Алкоголь_в_напитке'] = ['нет', 'да', 'да']
df_test['Уровень_красноречия'] = ['средний', 'высокий', 'средний']
df_test['Потраченные_деньги'] = ['много', 'мало', 'много']
df_test = create_df(df_test, features)
df_test

,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,False,True,False,True,False,True,False,True
1,False,True,True,False,True,False,True,False
2,True,False,True,False,False,True,False,True


In [6]:
# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
y = df_train['Поедет']
df_train, df_test = intersect_features(train=df_train, test=df_test)
df_train

,Потраченные_деньги_много,Потраченные_деньги_мало,Алкоголь_в_напитке_нет,Алкоголь_в_напитке_да,Уровень_красноречия_высокий,Внешность_приятная,Внешность_отталкивающая,Уровень_красноречия_средний
0,True,False,False,True,True,True,False,False
1,False,True,False,True,False,True,False,False
2,True,False,True,False,False,True,False,True
3,False,True,True,False,False,False,True,True
4,True,False,False,True,False,False,True,False
5,True,False,False,True,True,False,True,False
6,True,False,False,True,False,True,False,True


In [7]:
df_test

,Потраченные_деньги_много,Потраченные_деньги_мало,Алкоголь_в_напитке_нет,Алкоголь_в_напитке_да,Уровень_красноречия_высокий,Внешность_приятная,Внешность_отталкивающая,Уровень_красноречия_средний
0,True,False,True,False,False,True,False,True
1,False,True,False,True,True,True,False,False
2,True,False,False,True,False,False,True,True


Постройте от руки (или в графическом редакторе) дерево решений для этого набора данных. Дополнительно (для желающих) – можете сделать отрисовку дерева и написать код для построения всего дерева.

Вопрос 1. Какова энтропия начальной системы (S0
)? Под состояниями системы понимаем значения признака "Поедет" – 0 или 1 (то есть всего 2 состояния).

Вопрос 2. Рассмотрим разбиение обучающей выборки по признаку "Внешность_приятная". Какова энтропия S1
 левой группы, тех, у кого внешность приятная, и правой группы – S2
? Каков прирост информации при данном разбиении (IG)?


In [8]:
def shannon_entropy(data):
    # Подсчет количества каждого значения
    _, counts = np.unique(data, return_counts=True)
    
    # Вычисление вероятностей появления каждого значения
    probabilities = counts / len(data)
    
    # Вычисление энтропии по формуле Шеннона
    entropy = -np.sum(probabilities * np.log2(probabilities))
    
    return entropy

In [9]:
S0 = shannon_entropy(y)
S0

0.9852281360342515

In [10]:
S2 = shannon_entropy(pd.concat([y, df_train['Внешность_приятная']], axis=1)
                .query('Внешность_приятная == False')['Поедет'])
print(S2.round(3))
S1 = shannon_entropy(pd.concat([y, df_train['Внешность_приятная']], axis=1)
                .query('Внешность_приятная == True')['Поедет'])

0.918


In [11]:
(S0 - 4/7*S1 - 3/7*S2).round(3)

0.128

Постройте с помощью sklearn дерево решений, обучив его на обучающей выборке. Глубину можно не ограничивать.

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

In [13]:
clf_tree = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=17)
# max_depth – максимальная глубина дерева
# max_features — максимальное число признаков, по которым ищется лучшее разбиение в дереве (это нужно потому, что при большом количестве признаков будет "дорого" искать лучшее (по критерию типа прироста информации) разбиение среди всех признаков)
# min_samples_leaf – минимальное число объектов в листе. У этого параметра есть понятная интерпретация: скажем, если он равен 5, то дерево будет порождать только те классифицирующие правила, которые верны как минимум для 5 объектов


# обучаем дерево
clf_tree.fit(df_train, y)

export_graphviz(clf_tree, feature_names=df_train.keys(), 
out_file='/Users/asav/Documents/notebooks/roman_practice/Open_Data_Science/age_tree.dot', filled=True)
!dot -Tpng '/Users/asav/Documents/notebooks/roman_practice/Open_Data_Science/age_tree.dot' -o '/Users/asav/Documents/notebooks/roman_practice/Open_Data_Science/homework_tree.png'

In [14]:
clf_tree.predict(df_test)

array([0, 1, 1])

Часть 2. Функции для расчета энтропии и прироста информации
Примерчик для проверки: 9 синих шариков и 11 желтых. Пусть шарик находится в состоянии "1", если он синий и "0" – если он желтый.

In [15]:
balls = [1 for i in range(9)] + [0 for i in range(11)]
balls

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [16]:
# две группы
balls_left  = [1 for i in range(8)] + [0 for i in range(5)] # 8 синих и 5 желтых
balls_right = [1 for i in range(1)] + [0 for i in range(6)] # 1 синий и 6 желтых
print(balls_left)
print()
print(balls_right)

[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

[1, 0, 0, 0, 0, 0, 0]


In [17]:
print(shannon_entropy(balls)) # 9 синих и 11 желтых
print(shannon_entropy(balls_left)) # 8 синих и 5 желтых
print(shannon_entropy(balls_right)) # 1 синий и 6 желтых
print(shannon_entropy([1,2,3,4,5,6])) # энтропия игральной кости с несмещенным центром тяжести

0.9927744539878083
0.9612366047228759
0.5916727785823275
2.584962500721156


In [18]:
def information_gain(root, left, right):
    ''' root - изначальный набор данных, left и right два разбиения изначального набора'''
    return (shannon_entropy(root) - len(left)/len(root)*shannon_entropy(left)
            - len(right)/len(root)*shannon_entropy(right)) 

In [19]:
information_gain(balls, balls_left, balls_right)

0.16088518841412436

Описание признаков:

Age – возраст, количественный признак\
Workclass – тип работодателя, количественный признак\
fnlwgt – итоговый вес обьекта, количественный признак\
Education – уровень образования, качественный признак\
Education_Num – количество лет обучения, количественный признак\
Martial_Status – семейное положение, категориальный признак\
Occupation – профессия, категориальный признак\
Relationship – тип семейных отношений, категориальный признак\
Race – раса, категориальный признак\
Sex – пол, качественный признак\
Capital_Gain – прирост капитала, количественный признак\
Capital_Loss – потери капитала, количественный признак\
Hours_per_week – количество часов работы в неделю, количественный признак\
Country – страна, категориальный признак\

Целевая переменная: Target – уровень заработка, категориальный (бинарный) признак



In [20]:
data_train = pd.read_csv(
    '/Users/asav/Documents/notebooks/roman_practice/Open_Data_Science/adult_train.csv', sep=';'
) 

In [21]:
data_train.tail()

,Age,Workclass,fnlwgt,Education,Education_Num,Martial_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_per_week,Country,Target
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [22]:
data_test = pd.read_csv(
    '/Users/asav/Documents/notebooks/roman_practice/Open_Data_Science/adult_test.csv', sep=';'
) 

In [23]:
data_test.tail()

,Age,Workclass,fnlwgt,Education,Education_Num,Martial_Status,Occupation,Relationship,Race,Sex,Capital_Gain,Capital_Loss,Hours_per_week,Country,Target
16277,39,Private,215419.0,Bachelors,13.0,Divorced,Prof-specialty,Not-in-family,White,Female,0.0,0.0,36.0,United-States,<=50K.
16278,64,NaN,321403.0,HS-grad,9.0,Widowed,NaN,Other-relative,Black,Male,0.0,0.0,40.0,United-States,<=50K.
16279,38,Private,374983.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
16280,44,Private,83891.0,Bachelors,13.0,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455.0,0.0,40.0,United-States,<=50K.
16281,35,Self-emp-inc,182148.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,60.0,United-States,>50K.


In [24]:
# # необходимо убрать строки с неправильными метками в тестовой выборке
data_test = data_test[(data_test['Target'] == ' >50K.') 
                      | (data_test['Target']==' <=50K.')]

In [25]:
# # перекодируем target в числовое поле
data_train.loc[data_train['Target'] == ' >50K', "Target"]  = 1
data_train.loc[data_train['Target'] == ' <=50K', "Target"] = 0

data_test.loc[data_test['Target'] == ' >50K.', 'Target'] = 1
data_test.loc[data_test['Target'] == ' <=50K.', 'Target'] = 0

In [26]:
data_test.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Age,16281,73,35,461,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Workclass,15318,8,Private,11210,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fnlwgt,16281.0,NaN,NaN,NaN,189435.677784,105714.907671,13492.0,116736.0,177831.0,238384.0,1490400.0
Education,16281,16,HS-grad,5283,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Education_Num,16281.0,NaN,NaN,NaN,10.072907,2.567545,1.0,9.0,10.0,12.0,16.0
Martial_Status,16281,7,Married-civ-spouse,7403,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Occupation,15315,14,Prof-specialty,2032,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Relationship,16281,6,Husband,6523,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Race,16281,5,White,13946,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,16281,2,Male,10860,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
data_train['Target'].value_counts()

Target
0    24720
1     7841
Name: count, dtype: int64

In [28]:
# fig = plt.figure(figsize=(25, 15))
# cols = 5
# rows = np.ceil(float(data_train.shape[1]) / cols)
# for i, column in enumerate(data_train.columns):
#     ax = fig.add_subplot(rows, cols, i + 1)
#     ax.set_title(column)
#     if data_train.dtypes[column] == np.object:
#         data_train[column].value_counts().plot(kind="bar", axes=ax)
#     else:
#         data_train[column].hist(axes=ax)
#         plt.xticks(rotation="vertical")
# plt.subplots_adjust(hspace=0.7, wspace=0.2)


In [29]:
data_train.dtypes

Age                int64
Workclass         object
fnlwgt             int64
Education         object
Education_Num      int64
Martial_Status    object
Occupation        object
Relationship      object
Race              object
Sex               object
Capital_Gain       int64
Capital_Loss       int64
Hours_per_week     int64
Country           object
Target            object
dtype: object

In [30]:
data_test.dtypes

Age                object
Workclass          object
fnlwgt            float64
Education          object
Education_Num     float64
Martial_Status     object
Occupation         object
Relationship       object
Race               object
Sex                object
Capital_Gain      float64
Capital_Loss      float64
Hours_per_week    float64
Country            object
Target             object
dtype: object

In [31]:
data_test['Age'] = data_test['Age'].astype(int)

In [32]:
data_test['fnlwgt'] = data_test['fnlwgt'].astype(int)
data_test['Education_Num'] = data_test['Education_Num'].astype(int)
data_test['Capital_Gain'] = data_test['Capital_Gain'].astype(int)
data_test['Capital_Loss'] = data_test['Capital_Loss'].astype(int)
data_test['Hours_per_week'] = data_test['Hours_per_week'].astype(int)

In [33]:
# выделим в выборках категориальные и числовые поля

categorical_columns_train = [c for c in data_train.columns 
                             if data_train[c].dtype.name == 'object']
numerical_columns_train = [c for c in data_train.columns 
                           if data_train[c].dtype.name != 'object']

categorical_columns_test = [c for c in data_test.columns 
                            if data_test[c].dtype.name == 'object']
numerical_columns_test = [c for c in data_test.columns 
                          if data_test[c].dtype.name != 'object']

print('categorical_columns_test:', categorical_columns_test)
print('categorical_columns_train:', categorical_columns_train)
print('numerical_columns_test:', numerical_columns_test)
print('numerical_columns_train:', numerical_columns_train)

categorical_columns_test: ['Workclass', 'Education', 'Martial_Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country', 'Target']
categorical_columns_train: ['Workclass', 'Education', 'Martial_Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country', 'Target']
numerical_columns_test: ['Age', 'fnlwgt', 'Education_Num', 'Capital_Gain', 'Capital_Loss', 'Hours_per_week']
numerical_columns_train: ['Age', 'fnlwgt', 'Education_Num', 'Capital_Gain', 'Capital_Loss', 'Hours_per_week']


In [34]:
# заполним пропуски

for c in categorical_columns_train:
    data_train[c] = data_train[c].fillna(data_train[c].mode())
for c in categorical_columns_test:
    data_test[c] = data_test[c].fillna(data_train[c].mode())
    
for c in numerical_columns_train:
    data_train[c] = data_train[c].fillna(data_train[c].median())
for c in numerical_columns_test:
    data_test[c] = data_test[c].fillna(data_train[c].median())   

In [35]:
data_train = pd.concat([data_train, pd.get_dummies(data_train['Workclass'], 
                                                   prefix="Workclass"),
                      pd.get_dummies(data_train['Education'], prefix="Education"),
                      pd.get_dummies(data_train['Martial_Status'], prefix="Martial_Status"),
                      pd.get_dummies(data_train['Occupation'], prefix="Occupation"),
                      pd.get_dummies(data_train['Relationship'], prefix="Relationship"),
                      pd.get_dummies(data_train['Race'], prefix="Race"),
                      pd.get_dummies(data_train['Sex'], prefix="Sex"),
                      pd.get_dummies(data_train['Country'], prefix="Country")],
                     axis=1)

data_test = pd.concat([data_test, pd.get_dummies(data_test['Workclass'], prefix="Workclass"),
                      pd.get_dummies(data_test['Education'], prefix="Education"),
                      pd.get_dummies(data_test['Martial_Status'], prefix="Martial_Status"),
                      pd.get_dummies(data_test['Occupation'], prefix="Occupation"),
                      pd.get_dummies(data_test['Relationship'], prefix="Relationship"),
                      pd.get_dummies(data_test['Race'], prefix="Race"),
                      pd.get_dummies(data_test['Sex'], prefix="Sex"),
                      pd.get_dummies(data_test['Country'], prefix="Country")],
                     axis=1)

In [36]:
data_train.drop(['Workclass', 'Education', 'Martial_Status',
                 'Occupation', 'Relationship', 'Race', 'Sex', 'Country'],
                axis=1, inplace=True)
data_test.drop(['Workclass', 'Education', 'Martial_Status', 'Occupation', 
                'Relationship', 'Race', 'Sex', 'Country'],
               axis=1, inplace=True)

In [37]:
data_test.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Age,16281.0,NaN,NaN,NaN,38.767459,13.849187,17.0,28.0,37.0,48.0,90.0
fnlwgt,16281.0,NaN,NaN,NaN,189435.677784,105714.907671,13492.0,116736.0,177831.0,238384.0,1490400.0
Education_Num,16281.0,NaN,NaN,NaN,10.072907,2.567545,1.0,9.0,10.0,12.0,16.0
Capital_Gain,16281.0,NaN,NaN,NaN,1081.905104,7583.935968,0.0,0.0,0.0,0.0,99999.0
Capital_Loss,16281.0,NaN,NaN,NaN,87.899269,403.105286,0.0,0.0,0.0,0.0,3770.0
...,...,...,...,...,...,...,...,...,...,...,...
Country_ Thailand,16281,2,False,16269,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country_ Trinadad&Tobago,16281,2,False,16273,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country_ United-States,16281,2,True,14662,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country_ Vietnam,16281,2,False,16262,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
set(data_train.columns) - set(data_test.columns)

{'Country_ Holand-Netherlands'}

In [39]:
data_train.shape, data_test.shape

((32561, 106), (16281, 105))

In [40]:
data_test['Country_ Holand-Netherlands'] = np.zeros([data_test.shape[0], 1])

In [41]:
[data_test.shape[0], 1]

[16281, 1]

In [42]:
set(data_train.columns) - set(data_test.columns)

set()

In [43]:
data_train.head(2)

,Age,fnlwgt,Education_Num,Capital_Gain,Capital_Loss,Hours_per_week,Target,Workclass_ Federal-gov,Workclass_ Local-gov,Workclass_ Never-worked,...,Country_ Portugal,Country_ Puerto-Rico,Country_ Scotland,Country_ South,Country_ Taiwan,Country_ Thailand,Country_ Trinadad&Tobago,Country_ United-States,Country_ Vietnam,Country_ Yugoslavia
0,39,77516,13,2174,0,40,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,50,83311,13,0,0,13,0,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [44]:
data_test.head(2)

,Age,fnlwgt,Education_Num,Capital_Gain,Capital_Loss,Hours_per_week,Target,Workclass_ Federal-gov,Workclass_ Local-gov,Workclass_ Never-worked,...,Country_ Puerto-Rico,Country_ Scotland,Country_ South,Country_ Taiwan,Country_ Thailand,Country_ Trinadad&Tobago,Country_ United-States,Country_ Vietnam,Country_ Yugoslavia,Country_ Holand-Netherlands
1,25,226802,7,0,0,40,0,False,False,False,...,False,False,False,False,False,False,True,False,False,0.0
2,38,89814,9,0,0,50,0,False,False,False,...,False,False,False,False,False,False,True,False,False,0.0


In [45]:
X_train = data_train.drop(['Target'], axis=1)
y_train = data_train['Target']

# X_test=data_test.drop(['Target'], axis=1)
# y_test = data_test['Target']

In [46]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=17)
# max_depth – максимальная глубина дерева
# max_features — максимальное число признаков, по которым ищется лучшее разбиение в дереве (это нужно потому, что при большом количестве признаков будет "дорого" искать лучшее (по критерию типа прироста информации) разбиение среди всех признаков)
# min_samples_leaf – минимальное число объектов в листе. У этого параметра есть понятная интерпретация: скажем, если он равен 5, то дерево будет порождать только те классифицирующие правила, которые верны как минимум для 5 объектов


# обучаем дерево
tree.fit(X_train, y_train)

export_graphviz(tree, feature_names=X_train.keys(), 
out_file='/Users/asav/Documents/notebooks/roman_practice/Open_Data_Science/adult_tree.dot', filled=True)
!dot -Tpng '/Users/asav/Documents/notebooks/roman_practice/Open_Data_Science/adult_tree.dot' -o '/Users/asav/Documents/notebooks/roman_practice/Open_Data_Science/adult_tree.png'

In [47]:
import warnings
warnings.filterwarnings("ignore")


new_data_test = pd.DataFrame()
for col in data_train.columns:
    new_data_test[col] = data_test[col]

X_test = new_data_test.drop(['Target'], axis=1)
y_test = new_data_test['Target']


tree_predictions = tree.predict(X_test)

In [48]:
(tree_predictions == y_test.values).mean()

0.8447884036607088

In [49]:
accuracy = accuracy_score(tree_predictions, y_test.values)

In [50]:
accuracy

0.8447884036607088

In [51]:
tree_2 = DecisionTreeClassifier(criterion='entropy', max_depth=9, random_state=17)

tree_2.fit(X_train, y_train)


DecisionTreeClassifier(criterion='entropy', max_depth=9, random_state=17)

In [52]:
tree_predictions_2 = tree_2.predict(X_test)

In [53]:
accuracy_2 = accuracy_score(y_test.values, tree_predictions_2)

In [54]:
accuracy_2

0.8565198697868681